In [141]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import datetime
now = datetime.datetime.now()

In [199]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}
year = 2018 # Year we're currently testing.
html = requests.get('https://en.wikipedia.org/wiki/List_of_Billboard_Hot_100_top-ten_singles_in_' + str(year),cookies={ 'sessionid': '123..'},headers=headers).text
soup = BeautifulSoup(html, 'lxml')
print(soup.prettify()[0:1000])

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of Billboard Hot 100 top-ten singles in 2018 - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesShort":["","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"],"wgRequestId":"XgF-PwpAAEMAAJZxPJAAAADF","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_Billboard_Hot_100_top-ten_singles_in_2018","wgTitle":"List of Billboard Hot 100 top-ten singles in 2018","wgCurRevisionId":932137003,"wgRevisionId":932137003,"wgArticleId":56098454,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":

In [200]:
my_table = soup.find('table',{'class':'wikitable sortable'})
type(my_table)

bs4.element.Tag

In [201]:
tableheaderincluded = True
swiftcount = 0 # Keeps track of where repeat artists with rowspans greater than 1 should be inserted.
drakecontingency = 0 # Used to keep track of how many rows have the same artist when the rowspan is greater than 1.

# The following code mines the first table found on the Wikipedia page and extracts only table elements that do not correspond to a month and date. We don't care about the exact month and day that the song achieved Top 10 glory or their peek date. Furthermore, the way Wikipedia's tables are formatted, there are occasionally multiple songs that were all entered on the same day, and that date is listed only once, which creates problems for our script later down the road.

In [202]:
Titles = []
try:
    tds = my_table.findAll('td')
except AttributeError:
    my_table = soup.find('table',{'class':'wikitable'})
    tableheaderincluded = False
    tds = my_table.findAll('td')

for td in tds:
    try:
        d = datetime.datetime.strptime(td.get_text()[:-1], "%B %d") # Check if the current string is a Month Day format.
    except ValueError:
        try:
            d = datetime.datetime.strptime(td.get_text()[:td.get_text().rfind('(')-1], "%B %d") # Catch a rare case where a song from the previous year charted again.
        except ValueError:
            try:
                d = datetime.datetime.strptime(td.get_text()[:td.get_text().rfind('[')-1], "%B %d") # Catch a rare case where a song's top ten entry date has an annotation.
            except ValueError:
                try:
                    if td.attrs['rowspan']:
                        taylorswift = td.get_text()[:-1]
                        swiftcount = 5
                        drakecontingency = int(td.attrs['rowspan']) - 1
                except KeyError:
                    pass
                if td.get_text()[0] != '[' and td.get_text()[0] and td.get_text()[0].strip(): # Drop annotations column.
                    if swiftcount == 1:
                        drakecontingency = drakecontingency-1;
                        print(taylorswift + ' appended!')
                        Titles.append(taylorswift)
                        if drakecontingency == 0:
                            taylorswift = ''
                            swiftcount = 0
                        else:
                            swiftcount = 4;
                    if swiftcount > 0:
                        swiftcount = swiftcount-1;
                    Titles.append(td.get_text()[:-1]) # Drop the newline character present at the end of every single entry.

Drake appended!
J. Cole appended!
J. Cole appended!
Drake appended!
Drake appended!
Drake appended!
Travis Scott appended!


In [203]:
print(Titles)

['"Havana" (#4)', 'Camila Cabello featuring Young Thug', '1', '23', '"No Limit"', 'G-Eazy featuring ASAP Rocky and Cardi B', '4', '9', '"Bad at Love"', 'Halsey', '5', '10', '"Finesse"', 'Bruno Mars and Cardi B', '3', '14', '"Filthy" ◁', 'Justin Timberlake', '9', '1', '"New Rules"', 'Dua Lipa', '6', '5', '"God\'s Plan" (#1) ◁', 'Drake', '1', '26', '"Diplomatic Immunity" ◁', 'Drake', '7', '1', '"Say Something" ◁', 'Justin Timberlake featuring Chris Stapleton', '9', '1', '"Pray for Me"[C] ◁', 'The Weeknd and Kendrick Lamar', '7', '6', '"Stir Fry"[D]', 'Migos', '8', '4', '"Meant to Be" (#3) [J][L]', 'Bebe Rexha featuring Florida Georgia Line', '2', '19', '"Look Alive"[F] ◁', 'BlocBoy JB featuring Drake', '5', '13', '"All the Stars"[E]', 'Kendrick Lamar and SZA', '7', '4', '"Psycho" (#6) [M] ◁', 'Post Malone featuring Ty Dolla Sign', '1', '21', '"The Middle" (#8)', 'Zedd, Maren Morris and Grey', '5', '16', '"Sad!"[K]', 'XXXTentacion', '1', '6', '"Freaky Friday" ◁', 'Lil Dicky featuring Chri

In [204]:
nptitles = np.asarray(Titles).copy()

In [205]:
len(nptitles)

256

In [206]:
numsongs = int(np.prod(nptitles.shape)/4) # Find out how many songs are in this list.
reshaped = nptitles.reshape(numsongs,4) # Reshape the array based on the number of songs and the number of columns.

In [207]:
songdata = pd.DataFrame(reshaped)
songdata.columns = ['Title','Artist','Peak','Number of weeks in top ten']
songdata['Year'] = str(year)
songdata['Title'] = [i[1:i.rfind('\"')] for i in songdata['Title']] # Drops quotation marks and annotations in Title.
cleanweeks = []
if year == now.year:
    for i in songdata['Number of weeks in top ten']: # This code searches the number of weeks in top ten column for asterisks and eliminates them. This is only a 
        try:                                         # problem in the current year.
            cleanweeks.append(int(i[:i.rindex('*')]))
        except ValueError:
            cleanweeks.append(int(i))

    songdata['Number of weeks in top ten'] = cleanweeks

In [208]:
songdata

,Title,Artist,Peak,Number of weeks in top ten,Year
0,Havana,Camila Cabello featuring Young Thug,1,23,2018
1,No Limit,G-Eazy featuring ASAP Rocky and Cardi B,4,9,2018
2,Bad at Love,Halsey,5,10,2018
3,Finesse,Bruno Mars and Cardi B,3,14,2018
4,Filthy,Justin Timberlake,9,1,2018
...,...,...,...,...,...
59,Mia,Bad Bunny featuring Drake,5,1,2018
60,Mo Bamba,Sheck Wes,6,6,2018
61,"Thank U, Next",Ariana Grande,1,17,2018
62,Going Bad,Meek Mill featuring Drake,6,2,2018


In [188]:
songdata.to_csv(path_or_buf='data/' + str(year) + 'top10songs.csv',index=False) # Export our sample data table.